In [1]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

# import ind.alpha191 as a191



from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates


# from QUANTAXIS.QAUtil import DATABASE
# from QUANTAXIS.QAUtil import  trade_date_sse
# from QUANTAXIS.QAUtil.QADate_trade import (
#     QA_util_get_pre_trade_date,
#     QA_util_get_next_trade_date,
#     QA_util_if_tradetime
# )

# import inspect

# from sklearn import linear_model

%load_ext autoreload
%autoreload 2
%aimport ind.alpha191

In [2]:
# # a = load_cache('alpha11_train', cache_type=CACHE_TYPE.FACTOR)
# # b = load_cache('alpha24_train', cache_type=CACHE_TYPE.FACTOR)
# pd.concat([a,b],axis=1).sort_index().loc[(slice('2017-01-20'),['000001','000002']),:]

ids = np.arange(1,192)
# np.random.shuffle(ids)
files = list(map(lambda x:'191_alpha'+'0'*(3- int(np.log10(x)+1))+str(x)+'_train',ids))

def load_alpha_factors(file_names):
    from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)
    from base.JuUnits import excute_for_multidates
    import numpy as np
    import pandas as pd 
    
    temp = []
    for file in file_names:
        print(file)
        temp.append(load_cache(file, cache_type=CACHE_TYPE.FACTOR))
    
    return pd.concat(temp,axis=1)
        
worker=6
task = task_chunk_split(files, worker)
results = parallal_task(worker, load_alpha_factors, task)
res = pd.concat(results,axis=1)

Now in the main code. Process name is: base.JuUnits
base.JuUnits, subpid:9704  pid:25024


  0%|          | 0/6 [00:00<?, ?it/s]

In [3]:
res

191_alpha001  191_alpha002  191_alpha003  191_alpha004  \
date       code                                                             
2017-01-04 000001           NaN           NaN           NaN            -1   
           000002           NaN           NaN           NaN            -1   
           000063           NaN           NaN           NaN            -1   
           000066           NaN           NaN           NaN            -1   
           000069           NaN           NaN           NaN            -1   
...                         ...           ...           ...           ...   
2021-12-30 603882      0.122712     -0.045194     -7.484876             1   
           603899     -0.778039     -1.085591     -1.875745            -1   
           603986     -0.183787     -0.897904     -7.931123             1   
           603993      0.128770     -0.214286           NaN            -1   
           605499      0.227079     -0.431462     -5.093278            -1   

                   191_alpha005  191_alpha006  191_alpha007  191_alpha008  \
date       code                                                             
2017-01-04 000001           NaN           NaN           NaN           NaN   
           000002           NaN           NaN           NaN           NaN   
           000063           NaN           NaN           NaN           NaN   
           000066           NaN           NaN           NaN           NaN   
           000069           NaN           NaN           NaN           NaN   
...                         ...           ...           ...           ...   
2021-12-30 603882     -0.320767        -0.875      1.006174     -8.767666   
           603899     -0.642857        -0.625      0.489120     -2.223721   
           603986      0.191741        -0.500      0.075562     -0.271289   
           603993     -0.575224        -0.625      0.160738      0.032031   
           605499     -0.441415        -0.750      0.217265     -2.318500   

                   191_alpha009  191_alpha010  ...  191_alpha182  \
date       code                                ...                 
2017-01-04 000001           NaN           NaN  ...           NaN   
           000002           NaN           NaN  ...           NaN   
           000063           NaN           NaN  ...           NaN   
           000066           NaN           NaN  ...           NaN   
           000069           NaN           NaN  ...           NaN   
...                         ...           ...  ...           ...   
2021-12-30 603882  1.441345e-04      0.781765  ...          0.65   
           603899  2.056967e-05      0.753562  ...          0.50   
           603986  6.574575e-05      0.932701  ...          0.75   
           603993 -1.366303e-09      0.600587  ...          0.70   
           605499 -6.312849e-05      0.157895  ...          0.70   

                   191_alpha183  191_alpha184  191_alpha185  191_alpha186  \
date       code                                                             
2017-01-04 000001           NaN           NaN -1.191816e-06           NaN   
           000002           NaN           NaN -2.783385e-05           NaN   
           000063           NaN           NaN -9.585063e-06           NaN   
           000066           NaN           NaN -3.121001e-05           NaN   
           000069           NaN           NaN -8.186639e-06           NaN   
...                         ...           ...           ...           ...   
2021-12-30 603882    126.760767      0.439884 -1.254828e-04     10.626238   
           603899     40.454674      0.918458 -4.799087e-04     10.274729   
           603986    116.214508      0.662072 -4.268908e-03     68.217443   
           603993     44.789130      0.735377 -3.270015e-06     11.560463   
           605499     94.339424           NaN -3.116473e-07     10.658312   

                   191_alpha187  191_alpha188  191_alpha189  191_alpha190  \
date       code                                    

In [29]:
# res.loc[(slice(None),'000001'),:].isna().sum(axis=0)

def nan_check(x):
    return round(x.isna().sum(axis=0)/len(x),4)*100
    
# def valid_rate_check(x):
    

# valid_rate = excute_for_multidates(res, nan_check, level=1)
# valid_distribute_boundary = valid_rate.mean(axis=0) + valid_rate.std(axis=0)*4

valid_distribute_boundary

# with pd.option_context('display.max_rows', None, "display.max_columns", None): 
#     display(valid_rate)

191_alpha001     4.688336
191_alpha002     4.295780
191_alpha003    20.266088
191_alpha004     0.000000
191_alpha005    16.729974
                  ...    
191_alpha187    14.121832
191_alpha188     3.414374
191_alpha189     7.059339
191_alpha190    26.830477
191_alpha191    16.264282
Length: 191, dtype: float64